In [ ]:
#### SetUp

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score, confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV, KFold, StratifiedKFold
from sklearn.linear_model import Lasso, LogisticRegression, RidgeClassifier
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import balanced_accuracy_score

import pywt
import numpy as np
import pandas as pd
import pywt
import seaborn as sns
import scaleogram as scg 
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
from mat4py import loadmat
from scipy.fftpack import fft
import seaborn as sn

import matplotlib.pyplot as plt
from scipy.stats import mode
from statsmodels.tsa.stattools import acf, pacf

from collections import Counter
from scipy import stats
from itertools import repeat

import biosppy.signals.ecg as ecg

vColors = ["#049DD9", "#03A64A", "#F2AC29", "#F2CA80", "#F22929"]

In [2]:
X_train = pd.read_csv("features_train.csv")
y_train = pd.read_csv("train_labels.csv").to_numpy()[:,1]

In [3]:
# Parameter Tuning

param_test = {'max_depth': [10, 15, 18],
              'min_child_samples': [6,8,10],
              'subsample': [0.7,0.8],
              'colsample_bytree': [0.8, 0.7]}

gsearch5 = GridSearchCV(estimator=lgb.LGBMClassifier(objective='multiclass',
                                                    num_iterations = 100,
                                                    learning_rate = 0.1,
                                                    num_threads = 4,
                                                    class_weight="balanced",
                                                    random_state = 42),
                        param_grid=param_test,
                        scoring="balanced_accuracy",
                        n_jobs= -1, 
                        cv=5)

gsearch5.fit(X_train, y_train)

pd.DataFrame(gsearch5.cv_results_).sort_values("rank_test_score")
print(gsearch5.best_params_)
print(gsearch5.best_score_)

In [7]:
pd.DataFrame(gsearch5.cv_results_).sort_values("rank_test_score")
print(gsearch5.best_params_)
print(gsearch5.best_score_)


{'colsample_bytree': 0.8, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.7}
0.9114251591323063


In [37]:
model = lgb.LGBMClassifier(objective='multiclass',
                        n_estimators = 60,
                        learning_rate = 0.1,
                        random_state=42,
                        max_depth = 150,
                        min_child_samples = 20,
                        subsample = 0.75,
                        colsample_bytree = 0.8,
                        reg_alpha = 0,
                        class_weight="balanced"
                        )


BAC = []
cv = KFold(n_splits=5)

for train_ix, test_ix in cv.split(X_train):

    X_cvtrain, X_cvtest = X_train.iloc[train_ix, :], X_train.iloc[test_ix, :]
    y_cvtrain, y_cvtest = y_train[train_ix], y_train[test_ix]

    model.fit(X_cvtrain, y_cvtrain)

    predtrain = model.predict(X_cvtrain)
    pred = model.predict(X_cvtest)

    print("\nTrain BMAC:")
    print(np.round(balanced_accuracy_score(y_cvtrain, predtrain), 4))
    print("\nTest BMAC:")
    print(np.round(balanced_accuracy_score(y_cvtest, pred), 4))
    print("\n________________________")

    BAC.append(np.round(balanced_accuracy_score(y_cvtest, pred), 4))

print("\nAverage BMAC:", round(np.sum(BAC)/5, 4))
print("Std:", round(np.std(BAC), 4))


Train BMAC:
0.9806

Test BMAC:
0.9272

________________________

Train BMAC:
0.98

Test BMAC:
0.9377

________________________

Train BMAC:
0.9754

Test BMAC:
0.9651

________________________

Train BMAC:
0.9742

Test BMAC:
0.9638

________________________

Train BMAC:
0.9795

Test BMAC:
0.938

________________________

Average BMAC: 0.9464
Std: 0.0153


In [ ]:
sn.heatmap(confusion_matrix(y_cvtest, pred), annot=True,cmap='Blues', fmt='g')

In [9]:
X_test = pd.read_csv("features_test.csv")

In [10]:
scaler = StandardScaler() 

X_train = pd.DataFrame(
    scaler.fit_transform(
    X_train),
    columns=X_train.columns)

X_test = pd.DataFrame(
    scaler.fit_transform(
    X_test),
    columns=X_test.columns)

In [31]:
model.fit(X_train, y_train)

preds = model.predict(X_test)

dfResults = pd.DataFrame({"Id": list(range(0, X_test.shape[0], 1)), "y": preds})

dfResults.to_csv("Results.csv", sep=',', index=False)